In [1]:
%load_ext rpy2.ipython

In [47]:
%%R
library(DESeq2)
library(magrittr)
library(SummarizedExperiment)

start_time <- Sys.time()

IN_DIR <- "/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/GEO_model_training/input/"
OUT_DIR <- "/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/GEO_model_training/output/"

GEO_model_training_DATA_DIR <- IN_DIR
GEO_model_training_DATA_FIL <- "rse_gene_80.Rdata"

ensembl2rxns.df <- read.table(paste(IN_DIR,"Ensembl2ReactomeReactions.txt",sep=""),
                              sep="\t")

load(paste(GEO_model_training_DATA_DIR,GEO_model_training_DATA_FIL,sep=""))

In [3]:
%%R
ls()

[1] "ensembl2rxns.df" "GEO_model_training_DATA_DIR"   "GEO_model_training_DATA_FIL"   "IN_DIR"         
[5] "OUT_DIR"         "rse_gene_80"     "start_time"     


In [48]:
%%R
tissue_data <- colData(rse_gene_80)
print(tissue_data)

DataFrame with 6322 rows and 203 columns
     external_id   rail_id       study sra.sample_acc.x sra.experiment_acc
     <character> <integer> <character>      <character>        <character>
1     SRR7072952   3222107   SRP143444       SRS3225311         SRX4003400
2     SRR6676157   3156720   SRP132264       SRS2917141         SRX3652684
3     SRR3490416    860340   SRP074626       SRS1429373         SRX1751970
4     SRR8474321   3695673   SRP181218       SRS4279756         SRX5279921
5     SRR8995994   6972717   SRP187076       SRS4708363         SRX5774875
...          ...       ...         ...              ...                ...
6318 SRR10399912   1478954   SRP228572       SRS5612333         SRX7100071
6319  SRR8474314   3695258   SRP181218       SRS4279749         SRX5279914
6320  SRR1510028   1442442   SRP044222        SRS656044          SRX648736
6321  SRR5121136   2762451   SRP095371       SRS1871356         SRX2436352
6322  SRR9179022   7080105   SRP200107       SRS4862343    

1                                   439160114
2                                 10538179325
3                                  1667626045
4                                  6903131063
5                                  1374073476
...                                       ...
6318                                241510263
6319                               6731931726
6320                                481096576
6321                               2567719574
6322                               4303348684
     recount_qc.bc_auc.unique_reads_annotated_bases recount_qc.bc_auc.all_%
                                          <numeric>               <numeric>
1                                         441808049                   98.37
2                                       10428043233                   98.45
3                                        1669066930                   98.57
4                                        6879927315                   98.90
5                                     

6321                                           0
6322                                           0
     recount_qc.star.%_of_reads_unmapped:_too_many_mismatches
                                                    <numeric>
1                                                           0
2                                                           0
3                                                           0
4                                                           0
5                                                           0
...                                                       ...
6318                                                        0
6319                                                        0
6320                                                        0
6321                                                        0
6322                                                        0
     recount_qc.star.%_of_reads_unmapped:_too_many_mismatches2
                                 

                                                   <integer>
1                                                    2811045
2                                                    4449707
3                                                    7913821
4                                                    5442200
5                                                    3508369
...                                                      ...
6318                                                   57035
6319                                                 3037114
6320                                                  435743
6321                                                 2524098
6322                                                 4411239
     recount_qc.star.number_of_reads_mapped_to_multiple_loci2
                                                    <integer>
1                                                           0
2                                                           0
3                   

4                                           0
5                                           0
...                                       ...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [51]:
%%R
tissue_data <- colData(rse_gene_80)

tissue_counts <- table(tissue_data$Major_tissue)

tissues_to_remove <- names(tissue_counts[tissue_counts <= 20])

rse_gene_80 <- rse_gene_80[, !(tissue_data$Major_tissue %in% c(tissues_to_remove, 'Spleen'))]

new_tissue_counts <- table(colData(rse_gene_80)$Major_tissue)
print(new_tissue_counts)

row_totals <- rowSums(assays(rse_gene_80)$raw_counts)

rse_gene_80 <- rse_gene_80[row_totals > 0, ]


      Adipose       Adrenal         Aorta          Bone         Brain 
          489            21            33            30          1341 
          Eye         Heart     Intestine        Kidney         Liver 
          121           160           463           360          1738 
         Lung Mammary gland           MOE        Muscle         Nerve 
          242            23           101           294            34 
    Olfactory         Ovary      Pancreas          Skin         Sperm 
           28            63           174           114            51 
  Spinal cord       Stomach        Tendon        Testes        Uterus 
           74            29            23           188            46 
          VNO 
           48 


In [52]:
%%R
new_tissue_counts_df <- as.data.frame(new_tissue_counts)
colnames(new_tissue_counts_df) <- c("Tissue", "Number")
write.csv(new_tissue_counts_df, paste0(OUT_DIR, "chart/tissue_counts.csv"), row.names = FALSE)

In [53]:
%%R
min(rowSums(assays(rse_gene_80)$raw_count))

[1] 1


In [54]:
%%R
GEO_model_training.cols <- rse_gene_80 %>% colData()
print(GEO_model_training.cols$Major_tissue)

   [1] "Liver"         "Brain"         "Intestine"     "Intestine"    
   [5] "Liver"         "Lung"          "Intestine"     "Liver"        
   [9] "Brain"         "Muscle"        "Liver"         "Liver"        
  [13] "Liver"         "Liver"         "Liver"         "Liver"        
  [17] "Brain"         "Muscle"        "Liver"         "Liver"        
  [21] "Testes"        "Heart"         "Skin"          "Brain"        
  [25] "Liver"         "Liver"         "Kidney"        "Testes"       
  [29] "Muscle"        "Skin"          "Intestine"     "Uterus"       
  [33] "Uterus"        "Liver"         "Muscle"        "Liver"        
  [37] "Brain"         "Sperm"         "Testes"        "Brain"        
  [41] "Eye"           "Liver"         "Intestine"     "Liver"        
  [45] "Liver"         "Adipose"       "Liver"         "Muscle"       
  [49] "Heart"         "Adipose"       "Lung"          "Eye"          
  [53] "Liver"         "Liver"         "Adipose"       "Adrenal"      
  [57]

 [921] "Brain"         "Muscle"        "Kidney"        "Brain"        
 [925] "Brain"         "Testes"        "Brain"         "Heart"        
 [929] "Liver"         "Adipose"       "Liver"         "Intestine"    
 [933] "Brain"         "Adipose"       "Liver"         "Brain"        
 [937] "Brain"         "Muscle"        "Liver"         "Muscle"       
 [941] "Liver"         "Bone"          "Heart"         "Brain"        
 [945] "Testes"        "Muscle"        "Eye"           "Liver"        
 [949] "Sperm"         "Liver"         "Uterus"        "Kidney"       
 [953] "Adipose"       "Intestine"     "Skin"          "Brain"        
 [957] "Kidney"        "Liver"         "Brain"         "Intestine"    
 [961] "Liver"         "Brain"         "VNO"           "Olfactory"    
 [965] "Liver"         "Liver"         "Brain"         "Sperm"        
 [969] "Brain"         "Liver"         "Liver"         "Liver"        
 [973] "Liver"         "Intestine"     "Intestine"     "Brain"        
 [977]

[1841] "Brain"         "Adipose"       "Sperm"         "Heart"        
[1845] "Ovary"         "Nerve"         "Heart"         "Brain"        
[1849] "Adipose"       "Brain"         "Intestine"     "Testes"       
[1853] "Eye"           "Liver"         "Nerve"         "Brain"        
[1857] "Liver"         "Muscle"        "Spinal cord"   "Lung"         
[1861] "Liver"         "Liver"         "Kidney"        "Liver"        
[1865] "Intestine"     "Adipose"       "Kidney"        "Brain"        
[1869] "Adipose"       "Intestine"     "Pancreas"      "Testes"       
[1873] "Liver"         "Ovary"         "Liver"         "Adipose"      
[1877] "Liver"         "Liver"         "MOE"           "Brain"        
[1881] "Brain"         "Kidney"        "Brain"         "Nerve"        
[1885] "Brain"         "Liver"         "Brain"         "Kidney"       
[1889] "Testes"        "Intestine"     "Liver"         "Stomach"      
[1893] "Intestine"     "Ovary"         "Adipose"       "Pancreas"     
[1897]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 "Tendon"        "Aorta"         "Muscle"        "Kidney"       
[4533] "Brain"         "Brain"         "Liver"         "Brain"        
[4537] "Kidney"        "Testes"        "Liver"         "Liver"        
[4541] "Brain"         "Muscle"        "Intestine"     "Eye"          
[4545] "Liver"         "Liver"         "Brain"         "Brain"        
[4549] "Brain"         "Liver"         "Liver"         "Intestine"    
[4553] "Brain"         "VNO"           "Lung"          "Brain"        
[4557] "Intestine"     "Brain"         "Pancreas"      "Intestine"    
[4561] "Intestine"     "Eye"           "Adipose"       "Brain"        
[4565] "Liver"         "Liver"         "Intestine"     "Liver"        
[4569] "Adipose"       "Brain"         "Adipose"       "Liver"        
[4573] "MOE"           "Muscle"        "Pancreas"      "Kidney"       
[4577] "Ovary"         "Brain"         "Kidney"        "Liver"        
[4581] "Kidney"        "Kidney"        "Skin"          "Kidney"       
[4585] "Eye"

[5449] "Intestine"     "Aorta"         "Pancreas"      "Brain"        
[5453] "Pancreas"      "Liver"         "Liver"         "Brain"        
[5457] "Brain"         "Brain"         "Liver"         "Liver"        
[5461] "Spinal cord"   "Liver"         "Heart"         "Intestine"    
[5465] "Intestine"     "Eye"           "Adipose"       "Adipose"      
[5469] "Liver"         "Liver"         "Liver"         "Liver"        
[5473] "Brain"         "Sperm"         "Liver"         "Brain"        
[5477] "Brain"         "Liver"         "Adipose"       "Liver"        
[5481] "Intestine"     "Liver"         "Kidney"        "Brain"        
[5485] "Liver"         "Brain"         "Lung"          "Brain"        
[5489] "Brain"         "Brain"         "Liver"         "Liver"        
[5493] "Liver"         "Pancreas"      "Adipose"       "VNO"          
[5497] "Liver"         "Brain"         "Brain"         "Liver"        
[5501] "Brain"         "Kidney"        "Brain"         "Adipose"      
[5505]

In [55]:
%%R
saveRDS(GEO_model_training.cols$Major_tissue,file=paste(OUT_DIR,"GEO_model_training_tissue_detail_vec.Rds",sep=""))

In [56]:
%%R
saveRDS(GEO_model_training.cols$Row.names,file=paste(OUT_DIR,"GEO_model_training_sample_detail_vec.Rds",sep=""))

In [60]:
%%R
GEO_model_training_df_col_data <- colData(rse_gene_80)
print(GEO_model_training_df_col_data$external_id)
sample_id_list <- GEO_model_training_df_col_data$external_id

   [1] "SRR7072952"  "SRR6676157"  "SRR3490416"  "SRR8474321"  "SRR8995994" 
   [6] "ERR1121538"  "SRR8327990"  "SRR9610521"  "SRR10400103" "SRR5273659" 
  [11] "SRR1810241"  "ERR2682244"  "SRR10108623" "SRR3156217"  "SRR6357612" 
  [16] "SRR5576370"  "SRR10399979" "SRR2887573"  "SRR7161669"  "SRR10108617"
  [21] "ERR2654382"  "ERR2391774"  "SRR2762387"  "SRR10399902" "SRR3156199" 
  [26] "SRR6653174"  "SRR9062577"  "ERR2654312"  "SRR945381"   "SRR7766578" 
  [31] "SRR6488930"  "DRR091793"   "SRR8514679"  "SRR2086300"  "SRR5074051" 
  [36] "ERR2704935"  "SRR4296493"  "ERR2012010"  "DRR022939"   "SRR7622937" 
  [41] "SRR4249668"  "SRR6221692"  "SRR7161285"  "SRR5464218"  "SRR5275569" 
  [46] "SRR1158539"  "SRR6423128"  "SRR2033430"  "ERR2220002"  "ERR1121500" 
  [51] "SRR207103"   "SRR5461518"  "SRR5275515"  "SRR6128391"  "SRR5641713" 
  [56] "SRR8474345"  "ERR2362946"  "SRR5998447"  "SRR8179974"  "SRR8474361" 
  [61] "SRR1205147"  "SRR10399944" "SRR8064877"  "SRR9182289"  "SRR1462352" 

[1061] "SRR8641072"  "SRR7161655"  "SRR7518325"  "SRR3161665"  "SRR9973557" 
[1066] "SRR2984117"  "SRR6488908"  "SRR7545306"  "SRR6737172"  "SRR10467111"
[1071] "SRR7138598"  "ERR2219985"  "SRR6423154"  "SRR7162054"  "SRR6781555" 
[1076] "SRR4216974"  "SRR10399941" "SRR8389931"  "SRR4017867"  "SRR10321920"
[1081] "SRR8444940"  "SRR10400013" "SRR2984060"  "ERR3417953"  "SRR594394"  
[1086] "SRR8114477"  "SRR7161666"  "SRR10400086" "DRR192181"   "SRR3129956" 
[1091] "ERR2287707"  "SRR8145370"  "SRR7161653"  "SRR6282673"  "ERR2216202" 
[1096] "SRR207098"   "SRR4292651"  "SRR3736970"  "SRR9264276"  "SRR5464241" 
[1101] "SRR9029903"  "SRR3156203"  "ERR2275553"  "SRR7161329"  "SRR7181101" 
[1106] "SRR3087172"  "SRR1462358"  "SRR7716710"  "DRR060878"   "SRR7161395" 
[1111] "SRR7161396"  "SRR9057210"  "ERR949491"   "SRR5435228"  "SRR6754137" 
[1116] "SRR1732347"  "ERR3417995"  "SRR2033421"  "SRR10467073" "SRR8474341" 
[1121] "ERR2588659"  "SRR449408"   "SRR594397"   "SRR8616783"  "ERR514731"  

[2121] "DRR142736"   "ERR2588682"  "SRR6467247"  "SRR3087152"  "SRR1181721" 
[2126] "SRR5277055"  "ERR2654304"  "SRR3987078"  "SRR8514697"  "SRR8156528" 
[2131] "SRR6221723"  "ERR2362945"  "SRR10523510" "SRR10400071" "SRR10108648"
[2136] "SRR5928848"  "SRR6221683"  "SRR1873549"  "SRR10467060" "DRR179028"  
[2141] "SRR7309740"  "SRR5275525"  "SRR3183061" 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 "SRR4842211"  "SRR6680394"  "ERR1624143" 
[4846] "SRR5641746"  "SRR5901918"  "SRR4000905"  "SRR5928872"  "SRR3182861" 
[4851] "SRR3714625"  "SRR7161675"  "SRR6225358"  "ERR1121539"  "SRR8554492" 
[4856] "ERR1121503"  "SRR6221706"  "SRR6907191"  "SRR8365900"  "SRR4216960" 
[4861] "SRR6714697"  "SRR9057192"  "SRR10523513" "SRR10467097" "SRR5280062" 
[4866] "ERR474000"   "SRR1691201"  "SRR7059334"  "SRR5273639"  "SRR6825180" 
[4871] "SRR7545304"  "SRR3156177"  "SRR6680466"  "SRR8114476"  "SRR6653160" 
[4876] "SRR1158594"  "SRR5448483"  "SRR1810253"  "SRR10242495" "SRR1958828" 
[4881] "SRR2762392"  "SRR7360467"  "SRR10400131" "SRR9302921"  "SRR10400089"
[4886] "ERR2654361"  "SRR3182856"  "ERR1121527"  "SRR7181111"  "SRR3161699" 
[4891] "SRR6884614"  "SRR6178915"  "SRR7161865"  "SRR8760541"  "SRR5973177" 
[4896] "SRR6653173"  "SRR8269424"  "SRR3322952"  "SRR7161670"  "SRR3171794" 
[4901] "SRR8951365"  "SRR10523486" "SRR5464188"  "SRR5273695"  "SRR5275571" 
[4906] "SRR3171437"  "ERR2362964"

[5901] "SRR9937465"  "ERR1121491"  "ERR2588654"  "SRR7771846"  "SRR10241200"
[5906] "SRR4043645"  "SRR10400157" "SRR5275513"  "SRR6680468"  "SRR10400129"
[5911] "SRR1158572"  "ERR1624144"  "SRR4043625"  "ERR2930264"  "ERR1082757" 
[5916] "SRR9621251"  "SRR2182298"  "SRR5656220"  "SRR8363851"  "SRR4281073" 
[5921] "SRR8757495"  "SRR10241193" "SRR10399985" "SRR9182332"  "SRR1810222" 
[5926] "SRR10067348" "SRR1209754"  "SRR9937471"  "SRR2728904"  "SRR8209791" 
[5931] "SRR8581336"  "SRR6680399"  "ERR949443"   "SRR5758620"  "ERR450636"  
[5936] "SRR5850848"  "SRR1158537"  "SRR5517152"  "SRR7161867"  "SRR8156534" 
[5941] "SRR8664484"  "ERR949440"   "SRR5275501"  "SRR8474362"  "SRR5641738" 
[5946] "SRR10399934" "SRR5487024"  "ERR2631466"  "SRR10241181" "SRR10399943"
[5951] "SRR6680576"  "ERR2704932"  "SRR1158573"  "SRR7309758"  "SRR8760537" 
[5956] "SRR10400030" "SRR5515084"  "SRR6221724"  "SRR5928879"  "SRR7161734" 
[5961] "ERR3771182"  "DRR075474"   "SRR6900420"  "SRR4043637"  "SRR3180544" 

In [61]:
%%R
GEO_model_training.df <- rse_gene_80 %>% SummarizedExperiment::assay() %>% as.data.frame()
print(GEO_model_training.df)

                       V1    V2    V3     V4   V5    V6     V7    V8 V9 V10
ENSMUSG00000079800.2    0     0   193      0    0   350      0     0  0   0
ENSMUSG00000095092.1    0     0     0      0    3     0      0     0  0   0
ENSMUSG00000079192.2    0     0     0    221    0    25      0    29  0   0
ENSMUSG00000079794.2    0     0    49      0    0     1      0   172  0   0
ENSMUSG00000094799.1    0     0     0      0    0    50      0     0  0   0
ENSMUSG00000095250.1    0     0   488      0    0   350      0    60  0   0
ENSMUSG00000095787.1    0     0     0    105    0     0      0   223  0   0
ENSMUSG00000096100.1    0     0    49      6    0    24      0    10  0  96
ENSMUSG00000094054.1    0     0     0      0    0   100      0     0  0   0
ENSMUSG00000095672.1    0     0    25      0    0     0      0     0  0   0
ENSMUSG00000079190.3  100     0   287    321    0  1298      0    19  0   0
ENSMUSG00000094514.1    0     0   245      0    0   490      0   549  0   0
ENSMUSG00000

ENSMUSG00000079800.2     0     0   761     0   154    0    0     0   214     0
ENSMUSG00000095092.1     0     0     0     0     0    0    0     0     0     0
ENSMUSG00000079192.2     0     0   374     0     0    0    0     0     0     0
ENSMUSG00000079794.2     0     0   196     0     0    0    0     0     0     0
ENSMUSG00000094799.1     0     0     0     0     0    0    0     0     0     0
ENSMUSG00000095250.1     0     0   117     0     0    0    0    76   111    73
ENSMUSG00000095787.1     0     0   303     0     0    0    0     0   186     0
ENSMUSG00000096100.1     0     0   420     0     0    0    0     0     0     0
ENSMUSG00000094054.1     0     0     0     0   123    0    0     0     0     0
ENSMUSG00000095672.1     0     0   138     0     0    0    0     0    61     0
ENSMUSG00000079190.3     0     0   702     0     0    0    0     0     0     0
ENSMUSG00000094514.1     0     0    74     0     0    0    0     0     0    76
ENSMUSG00000094915.1     0     0     0     0     0  

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     0    0    0     0     0     0    0
ENSMUSG00000096100.1   202    0    0     0    0    0     0   109     0    0
ENSMUSG00000094054.1     0    0    0   144    0    0     0    47     0    0
ENSMUSG00000095672.1     0    0    0     0    0    0     0     0     0  100
ENSMUSG00000079190.3     0    0    0   538    0    0    43   150     0  289
ENSMUSG00000094514.1     0    0    0    72    0   36     0    50     0    0
ENSMUSG00000094915.1     0    0    0     0    0    0     0     0     0    0
ENSMUSG00000079808.3     0    0    0   355  150    0   304   591   100  138
ENSMUSG00000063897.3 59955 1596 4608 37397 4962 1682 22901 20775 31063 2529
                      V396 V397 V398 V399  V400  V401 V402  V403  V404  V405
ENSMUSG00000079800.2     0  401    0    0   220   754    0     0     0     0
ENSMUSG00000095092.1     0    0    0    0     0     0    0     0     0     0
ENSMUSG00000079192.2     0   68    0    0     0     0    0    50     0     0
ENSMUSG00000079794.2     0    0    0    0   

ENSMUSG00000079190.3    908      0     0    0     0     0     0     0    0
ENSMUSG00000094514.1      0      0     0    0     0     0    76   168    0
ENSMUSG00000094915.1      0    380     0    0     0     0     0     0    0
ENSMUSG00000079808.3    286    276   238  121     0   571     0   126    0
ENSMUSG00000063897.3 125728 124509 32982   84 11458 47642 24362 22006 8195
                      V523  V524 V525 V526  V527  V528  V529  V530  V531  V532
ENSMUSG00000079800.2  1256  2346    0    0     0   183   160  2666     0     0
ENSMUSG00000095092.1   147     0    0    0     0     0     0     0     0     0
ENSMUSG00000079192.2   195   405    0    0     0   392     0   408    94     0
ENSMUSG00000079794.2     4    20    0    0     0     0     0   448    61     0
ENSMUSG00000094799.1     0     0    0    0     0    97     0     0     0     0
ENSMUSG00000095250.1   211   302    0    0     0    86   208     0     0    76
ENSMUSG00000095787.1     0   200    0    0     0     0     0     0     0

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     0  10060    51      0   840     0     0     0
ENSMUSG00000095092.1     0   266   1034     0      0   208    49     0     0
ENSMUSG00000079192.2     0     0   8484     0    255  4252     0   109     0
ENSMUSG00000079794.2     0    76   3378     0    136   160     0     0     0
ENSMUSG00000094799.1     0     0   1564     0      0   100     0     0     0
ENSMUSG00000095250.1     0   986   3089   153    204   487   148    85     0
ENSMUSG00000095787.1     0     0   3698     0      0  2245     0     0     0
ENSMUSG00000096100.1     0    76   9617     0    114  1196     0     0     0
ENSMUSG00000094054.1     0   152   1680   304    482  3094     0     0     0
ENSMUSG00000095672.1     0     0   3186     0    204   290     0    87     0
ENSMUSG00000079190.3     0   456  18172    51    714  3753     0     0     0
ENSMUSG00000094514.1   147  2152      0   214     75   471     0     0     0
ENSMUSG00000094915.1     0     0   4484    37    414   346     0     0     0
ENSMUSG00000079808.3   43

ENSMUSG00000094054.1     0     0   161      0     0   329     0    50   366
ENSMUSG00000095672.1     0     0     0      0   100     0   350     0     0
ENSMUSG00000079190.3     0     0    82    447    72   846     0    51   125
ENSMUSG00000094514.1     0     0     0      0     0     0     0   114     0
ENSMUSG00000094915.1     0     0     0    160     0   537   178     0   761
ENSMUSG00000079808.3     0   200   317    626     0   642   171   136     0
ENSMUSG00000063897.3     0 69567  8444 118205 36935 45481 59677  8374 96819
                     V5932 V5933 V5934 V5935 V5936 V5937 V5938 V5939 V5940
ENSMUSG00000079800.2   100     0     0   407     0     0     0     0  1069
ENSMUSG00000095092.1     0     0    52     0     0     0     0     0     0
ENSMUSG00000079192.2     0     0     0     0     0     0    77    62   142
ENSMUSG00000079794.2     0     0     0     0     0     0     0     0    50
ENSMUSG00000094799.1     0     0    52     0     0     0     0     0     0
ENSMUSG00000095250

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     0   302    74     0     0
ENSMUSG00000094915.1     0     0     0    39     0   694     0    93     0
ENSMUSG00000079808.3    96   231   102     0    89  1576   225   336   228
ENSMUSG00000063897.3  2673 28694 50374 41199 26355 24115 18366  2613 15134
                     V6193 V6194 V6195 V6196 V6197  V6198 V6199 V6200  V6201
ENSMUSG00000079800.2   200     0     0   100     0      0     0   101      0
ENSMUSG00000095092.1     0     0     0     0     0      0     0     0      0
ENSMUSG00000079192.2     0    33     0     0    81     84   195     0      0
ENSMUSG00000079794.2     0     0     0    46     0      4     4     0     43
ENSMUSG00000094799.1     0   225     0     0   200      0   100     0      0
ENSMUSG00000095250.1   100     0     0   100     0    527   198     0      0
ENSMUSG00000095787.1     0     0     0     0     0      0     0     0      0
ENSMUSG00000096100.1     0     0     0     0     0      0   226   101      0
ENSMUSG00000094054.1   100    72     0     0     0 

In [62]:
%%R
ensembl_wo_ids <- gsub("\\.[0-9]+","",rownames(GEO_model_training.df))
print(ensembl_wo_ids)

    [1] "ENSMUSG00000079800" "ENSMUSG00000095092" "ENSMUSG00000079192"
    [4] "ENSMUSG00000079794" "ENSMUSG00000094799" "ENSMUSG00000095250"
    [7] "ENSMUSG00000095787" "ENSMUSG00000096100" "ENSMUSG00000094054"
   [10] "ENSMUSG00000095672" "ENSMUSG00000079190" "ENSMUSG00000094514"
   [13] "ENSMUSG00000094915" "ENSMUSG00000079808" "ENSMUSG00000063897"
   [16] "ENSMUSG00000084520" "ENSMUSG00000096776" "ENSMUSG00000096244"
   [19] "ENSMUSG00000079222" "ENSMUSG00000094874" "ENSMUSG00000095450"
   [22] "ENSMUSG00000095500" "ENSMUSG00000094728" "ENSMUSG00000096808"
   [25] "ENSMUSG00000062783" "ENSMUSG00000090805" "ENSMUSG00000051412"
   [28] "ENSMUSG00000089163" "ENSMUSG00000061654" "ENSMUSG00000079834"
   [31] "ENSMUSG00000099278" "ENSMUSG00000094121" "ENSMUSG00000094576"
   [34] "ENSMUSG00000079764" "ENSMUSG00000079774" "ENSMUSG00000079773"
   [37] "ENSMUSG00000096271" "ENSMUSG00000094741" "ENSMUSG00000074720"
   [40] "ENSMUSG00000094337" "ENSMUSG00000095570" "ENSMUSG00000096728"
   [43

  [691] "ENSMUSG00000097899" "ENSMUSG00000102975" "ENSMUSG00000026073"
  [694] "ENSMUSG00000103560" "ENSMUSG00000026072" "ENSMUSG00000103230"
  [697] "ENSMUSG00000070942" "ENSMUSG00000026069" "ENSMUSG00000026070"
  [700] "ENSMUSG00000026068" "ENSMUSG00000104441" "ENSMUSG00000102676"
  [703] "ENSMUSG00000026065" "ENSMUSG00000103679" "ENSMUSG00000026062"
  [706] "ENSMUSG00000103270" "ENSMUSG00000041945" "ENSMUSG00000079588"
  [709] "ENSMUSG00000094136" "ENSMUSG00000104072" "ENSMUSG00000100185"
  [712] "ENSMUSG00000100832" "ENSMUSG00000100178" "ENSMUSG00000101877"
  [715] "ENSMUSG00000053640" "ENSMUSG00000101919" "ENSMUSG00000101299"
  [718] "ENSMUSG00000100180" "ENSMUSG00000103106" "ENSMUSG00000104199"
  [721] "ENSMUSG00000060424" "ENSMUSG00000045515" "ENSMUSG00000097063"
  [724] "ENSMUSG00000093436" "ENSMUSG00000099784" "ENSMUSG00000077448"
  [727] "ENSMUSG00000060679" "ENSMUSG00000041907" "ENSMUSG00000089088"
  [730] "ENSMUSG00000100269" "ENSMUSG00000041878" "ENSMUSG00000070939"
  [733

[11380] "ENSMUSG00000094533" "ENSMUSG00000095583" "ENSMUSG00000103271"
[11383] "ENSMUSG00000103297" "ENSMUSG00000096108" "ENSMUSG00000095642"
[11386] "ENSMUSG00000076661" "ENSMUSG00000104129" "ENSMUSG00000096805"
[11389] "ENSMUSG00000103918" "ENSMUSG00000094102" "ENSMUSG00000102515"
[11392] "ENSMUSG00000096459" "ENSMUSG00000076652" "ENSMUSG00000103121"
[11395] "ENSMUSG00000076666" "ENSMUSG00000103537" "ENSMUSG00000094029"
[11398] "ENSMUSG00000076668" "ENSMUSG00000103939" "ENSMUSG00000076670"
[11401] "ENSMUSG00000102479" "ENSMUSG00000102497" "ENSMUSG00000076672"
[11404] "ENSMUSG00000094322" "ENSMUSG00000102601" "ENSMUSG00000103989"
[11407] "ENSMUSG00000076674" "ENSMUSG00000104115" "ENSMUSG00000103340"
[11410] "ENSMUSG00000076671" "ENSMUSG00000076676" "ENSMUSG00000076677"
[11413] "ENSMUSG00000094174" "ENSMUSG00000096407" "ENSMUSG00000076680"
[11416] "ENSMUSG00000087582" "ENSMUSG00000102301" "ENSMUSG00000105438"
[11419] "ENSMUSG00000102524" "ENSMUSG00000095981" "ENSMUSG00000102641"
[11422

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 "ENSMUSG00000095821" "ENSMUSG00000114955"
[14110] "ENSMUSG00000090487" "ENSMUSG00000079101" "ENSMUSG00000079410"
[14113] "ENSMUSG00000091289" "ENSMUSG00000071613" "ENSMUSG00000090684"
[14116] "ENSMUSG00000079409" "ENSMUSG00000095512" "ENSMUSG00000096039"
[14119] "ENSMUSG00000096217" "ENSMUSG00000091849" "ENSMUSG00000079407"
[14122] "ENSMUSG00000096574" "ENSMUSG00000090318" "ENSMUSG00000095533"
[14125] "ENSMUSG00000096488" "ENSMUSG00000097382" "ENSMUSG00000072739"
[14128] "ENSMUSG00000090578" "ENSMUSG00000096905" "ENSMUSG00000094880"
[14131] "ENSMUSG00000095195" "ENSMUSG00000091010" "ENSMUSG00000095368"
[14134] "ENSMUSG00000079402" "ENSMUSG00000093891" "ENSMUSG00000090364"
[14137] "ENSMUSG00000096614" "ENSMUSG00000096450" "ENSMUSG00000095469"
[14140] "ENSMUSG00000096955" "ENSMUSG00000093887" "ENSMUSG00000097594"
[14143] "ENSMUSG00000097127" "ENSMUSG00000096218" "ENSMUSG00000096901"
[14146] "ENSMUSG00000096793" "ENSMUSG00000091756" "ENSMUSG00000091672"
[14149] "ENSMUSG00000095686" "ENSM

...

[40945] "ENSMUSG00000030235" "ENSMUSG00000030236" "ENSMUSG00000088865"
[40948] "ENSMUSG00000084927" "ENSMUSG00000030237" "ENSMUSG00000107516"
[40951] "ENSMUSG00000087904" "ENSMUSG00000041698" "ENSMUSG00000089616"
[40954] "ENSMUSG00000079263" "ENSMUSG00000092488" "ENSMUSG00000092500"
[40957] "ENSMUSG00000079262" "ENSMUSG00000088239" "ENSMUSG00000041681"
[40960] "ENSMUSG00000063975" "ENSMUSG00000041671" "ENSMUSG00000030243"
[40963] "ENSMUSG00000030245" "ENSMUSG00000071112" "ENSMUSG00000030244"
[40966] "ENSMUSG00000030246" "ENSMUSG00000030247" "ENSMUSG00000104234"
[40969] "ENSMUSG00000030249" "ENSMUSG00000030282" "ENSMUSG00000048473"
[40972] "ENSMUSG00000072683" "ENSMUSG00000030283" "ENSMUSG00000107443"
[40975] "ENSMUSG00000108085" "ENSMUSG00000107839" "ENSMUSG00000030279"
[40978] "ENSMUSG00000107693" "ENSMUSG00000107398" "ENSMUSG00000108094"
[40981] "ENSMUSG00000107802" "ENSMUSG00000030275" "ENSMUSG00000072679"
[40984] "ENSMUSG00000077241" "ENSMUSG00000084971" "ENSMUSG00000086887"
[40987

[41635] "ENSMUSG00000078798" "ENSMUSG00000081982" "ENSMUSG00000100566"
[41638] "ENSMUSG00000099596" "ENSMUSG00000074377" "ENSMUSG00000099744"
[41641] "ENSMUSG00000074375" "ENSMUSG00000084102" "ENSMUSG00000100378"
[41644] "ENSMUSG00000080705" "ENSMUSG00000098354" "ENSMUSG00000070810"
[41647] "ENSMUSG00000098506" "ENSMUSG00000099216" "ENSMUSG00000098782"
[41650] "ENSMUSG00000107673" "ENSMUSG00000030378" "ENSMUSG00000094156"
[41653] "ENSMUSG00000098290" "ENSMUSG00000098741" "ENSMUSG00000099242"
[41656] "ENSMUSG00000100586" "ENSMUSG00000099055" "ENSMUSG00000098520"
[41659] "ENSMUSG00000041596" "ENSMUSG00000093773" "ENSMUSG00000055942"
[41662] "ENSMUSG00000093665" "ENSMUSG00000105566" "ENSMUSG00000098330"
[41665] "ENSMUSG00000098508" "ENSMUSG00000093549" "ENSMUSG00000093664"
[41668] "ENSMUSG00000064977" "ENSMUSG00000093584" "ENSMUSG00000106552"
[41671] "ENSMUSG00000098309" "ENSMUSG00000098564" "ENSMUSG00000093736"
[41674] "ENSMUSG00000099198" "ENSMUSG00000098469" "ENSMUSG00000094555"
[41677

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 "ENSMUSG00000100919" "ENSMUSG00000101123" "ENSMUSG00000101276"
[54073] "ENSMUSG00000100231" "ENSMUSG00000101469" "ENSMUSG00000095508"
[54076] "ENSMUSG00000103330" "ENSMUSG00000104505" "ENSMUSG00000104468"
[54079] "ENSMUSG00000094135" "ENSMUSG00000103361" "ENSMUSG00000104267"
[54082] "ENSMUSG00000103352" "ENSMUSG00000102335" "ENSMUSG00000095769"
[54085] "ENSMUSG00000096820" "ENSMUSG00000103474" "ENSMUSG00000103468"
[54088] "ENSMUSG00000103176" "ENSMUSG00000104522" "ENSMUSG00000095462"
[54091] "ENSMUSG00000093903" "ENSMUSG00000102177" "ENSMUSG00000102346"
[54094] "ENSMUSG00000093883" "ENSMUSG00000095900" "ENSMUSG00000102174"
[54097] "ENSMUSG00000102583" "ENSMUSG00000104048" "ENSMUSG00000104404"
[54100] "ENSMUSG00000102799" "ENSMUSG00000102613" "ENSMUSG00000103524"
[54103] "ENSMUSG00000094227" "ENSMUSG00000103681" "ENSMUSG00000104409"
[54106] "ENSMUSG00000104127" "ENSMUSG00000103872" "ENSMUSG00000102787"
[54109] "ENSMUSG00000104499" "ENSMUSG00000103431" "ENSMUSG00000104191"
[54112] "ENSM

[54760] "ENSMUSG00000100167" "ENSMUSG00000100144" "ENSMUSG00000102103"
[54763] "ENSMUSG00000099527" "ENSMUSG00000101045" "ENSMUSG00000099920"
[54766] "ENSMUSG00000101021" "ENSMUSG00000099485" "ENSMUSG00000099548"
[54769] "ENSMUSG00000096875" "ENSMUSG00000099978" "ENSMUSG00000099595"
[54772] "ENSMUSG00000100539" "ENSMUSG00000100073" "ENSMUSG00000100223"
[54775] "ENSMUSG00000100772" "ENSMUSG00000101782" "ENSMUSG00000101155"
[54778] "ENSMUSG00000100295" "ENSMUSG00000101245" "ENSMUSG00000100578"
[54781] "ENSMUSG00000100456" "ENSMUSG00000096122" "ENSMUSG00000101943"
[54784] "ENSMUSG00000100724" "ENSMUSG00000099388" "ENSMUSG00000101399"
[54787] "ENSMUSG00000101350" "ENSMUSG00000100614" "ENSMUSG00000100083"
[54790] "ENSMUSG00000101991" "ENSMUSG00000100076" "ENSMUSG00000101100"
[54793] "ENSMUSG00000101744" "ENSMUSG00000095807" "ENSMUSG00000101317"
[54796] "ENSMUSG00000101244" "ENSMUSG00000096626" "ENSMUSG00000100085"
[54799] "ENSMUSG00000101424" "ENSMUSG00000100264" "ENSMUSG00000100715"
[54802

In [63]:
%%R
rownames(GEO_model_training.df) <- ensembl_wo_ids
print(rownames(GEO_model_training.df))

    [1] "ENSMUSG00000079800" "ENSMUSG00000095092" "ENSMUSG00000079192"
    [4] "ENSMUSG00000079794" "ENSMUSG00000094799" "ENSMUSG00000095250"
    [7] "ENSMUSG00000095787" "ENSMUSG00000096100" "ENSMUSG00000094054"
   [10] "ENSMUSG00000095672" "ENSMUSG00000079190" "ENSMUSG00000094514"
   [13] "ENSMUSG00000094915" "ENSMUSG00000079808" "ENSMUSG00000063897"
   [16] "ENSMUSG00000084520" "ENSMUSG00000096776" "ENSMUSG00000096244"
   [19] "ENSMUSG00000079222" "ENSMUSG00000094874" "ENSMUSG00000095450"
   [22] "ENSMUSG00000095500" "ENSMUSG00000094728" "ENSMUSG00000096808"
   [25] "ENSMUSG00000062783" "ENSMUSG00000090805" "ENSMUSG00000051412"
   [28] "ENSMUSG00000089163" "ENSMUSG00000061654" "ENSMUSG00000079834"
   [31] "ENSMUSG00000099278" "ENSMUSG00000094121" "ENSMUSG00000094576"
   [34] "ENSMUSG00000079764" "ENSMUSG00000079774" "ENSMUSG00000079773"
   [37] "ENSMUSG00000096271" "ENSMUSG00000094741" "ENSMUSG00000074720"
   [40] "ENSMUSG00000094337" "ENSMUSG00000095570" "ENSMUSG00000096728"
   [43

  [691] "ENSMUSG00000097899" "ENSMUSG00000102975" "ENSMUSG00000026073"
  [694] "ENSMUSG00000103560" "ENSMUSG00000026072" "ENSMUSG00000103230"
  [697] "ENSMUSG00000070942" "ENSMUSG00000026069" "ENSMUSG00000026070"
  [700] "ENSMUSG00000026068" "ENSMUSG00000104441" "ENSMUSG00000102676"
  [703] "ENSMUSG00000026065" "ENSMUSG00000103679" "ENSMUSG00000026062"
  [706] "ENSMUSG00000103270" "ENSMUSG00000041945" "ENSMUSG00000079588"
  [709] "ENSMUSG00000094136" "ENSMUSG00000104072" "ENSMUSG00000100185"
  [712] "ENSMUSG00000100832" "ENSMUSG00000100178" "ENSMUSG00000101877"
  [715] "ENSMUSG00000053640" "ENSMUSG00000101919" "ENSMUSG00000101299"
  [718] "ENSMUSG00000100180" "ENSMUSG00000103106" "ENSMUSG00000104199"
  [721] "ENSMUSG00000060424" "ENSMUSG00000045515" "ENSMUSG00000097063"
  [724] "ENSMUSG00000093436" "ENSMUSG00000099784" "ENSMUSG00000077448"
  [727] "ENSMUSG00000060679" "ENSMUSG00000041907" "ENSMUSG00000089088"
  [730] "ENSMUSG00000100269" "ENSMUSG00000041878" "ENSMUSG00000070939"
  [733

 [1381] "ENSMUSG00000099364" "ENSMUSG00000100562" "ENSMUSG00000079470"
 [1384] "ENSMUSG00000032883" "ENSMUSG00000099377" "ENSMUSG00000047330"
 [1387] "ENSMUSG00000101324" "ENSMUSG00000072978" "ENSMUSG00000099706"
 [1390] "ENSMUSG00000050711" "ENSMUSG00000099562" "ENSMUSG00000077782"
 [1393] "ENSMUSG00000102235" "ENSMUSG00000054702" "ENSMUSG00000104497"
 [1396] "ENSMUSG00000073643" "ENSMUSG00000026248" "ENSMUSG00000026249"
 [1399] "ENSMUSG00000067106" "ENSMUSG00000100397" "ENSMUSG00000099753"
 [1402] "ENSMUSG00000096297" "ENSMUSG00000102518" "ENSMUSG00000099552"
 [1405] "ENSMUSG00000043230" "ENSMUSG00000101257" "ENSMUSG00000103404"
 [1408] "ENSMUSG00000103076" "ENSMUSG00000103525" "ENSMUSG00000004364"
 [1411] "ENSMUSG00000090637" "ENSMUSG00000101722" "ENSMUSG00000110100"
 [1414] "ENSMUSG00000101483" "ENSMUSG00000103192" "ENSMUSG00000104488"
 [1417] "ENSMUSG00000102721" "ENSMUSG00000038608" "ENSMUSG00000099949"
 [1420] "ENSMUSG00000099918" "ENSMUSG00000054976" "ENSMUSG00000104029"
 [1423

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




 [4171] "ENSMUSG00000093298" "ENSMUSG00000111376" "ENSMUSG00000075225"
 [4174] "ENSMUSG00000102114" "ENSMUSG00000019813" "ENSMUSG00000111131"
 [4177] "ENSMUSG00000038332" "ENSMUSG00000071324" "ENSMUSG00000111103"
 [4180] "ENSMUSG00000085666" "ENSMUSG00000048756" "ENSMUSG00000088165"
 [4183] "ENSMUSG00000081873" "ENSMUSG00000083429" "ENSMUSG00000038302"
 [4186] "ENSMUSG00000087891" "ENSMUSG00000019804" "ENSMUSG00000019803"
 [4189] "ENSMUSG00000086434" "ENSMUSG00000038280" "ENSMUSG00000086428"
 [4192] "ENSMUSG00000087719" "ENSMUSG00000019802" "ENSMUSG00000111293"
 [4195] "ENSMUSG00000111333" "ENSMUSG00000111086" "ENSMUSG00000044770"
 [4198] "ENSMUSG00000111171" "ENSMUSG00000099893" "ENSMUSG00000102062"
 [4201] "ENSMUSG00000038248" "ENSMUSG00000045008" "ENSMUSG00000089673"
 [4204] "ENSMUSG00000110903" "ENSMUSG00000089839" "ENSMUSG00000110772"
 [4207] "ENSMUSG00000094955" "ENSMUSG00000038240" "ENSMUSG00000038214"
 [4210] "ENSMUSG00000045886" "ENSMUSG00000019797" "ENSMUSG00000111368"
 [421

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[44392] "ENSMUSG00000035165" "ENSMUSG00000109435" "ENSMUSG00000109186"
[44395] "ENSMUSG00000089987" "ENSMUSG00000030729" "ENSMUSG00000051048"
[44398] "ENSMUSG00000030718" "ENSMUSG00000109046" "ENSMUSG00000102241"
[44401] "ENSMUSG00000047248" "ENSMUSG00000099137" "ENSMUSG00000032942"
[44404] "ENSMUSG00000033685" "ENSMUSG00000030708" "ENSMUSG00000109560"
[44407] "ENSMUSG00000044881" "ENSMUSG00000109123" "ENSMUSG00000108990"
[44410] "ENSMUSG00000081359" "ENSMUSG00000030706" "ENSMUSG00000082210"
[44413] "ENSMUSG00000030704" "ENSMUSG00000030701" "ENSMUSG00000108866"
[44416] "ENSMUSG00000109035" "ENSMUSG00000097388" "ENSMUSG00000109109"
[44419] "ENSMUSG00000029461" "ENSMUSG00000008318" "ENSMUSG00000093296"
[44422] "ENSMUSG00000032875" "ENSMUSG00000109132" "ENSMUSG00000048779"
[44425] "ENSMUSG00000032860" "ENSMUSG00000108895" "ENSMUSG00000109171"
[44428] "ENSMUSG00000083121" "ENSMUSG00000111425" "ENSMUSG00000030691"
[44431] "ENSMUSG00000047767" "ENSMUSG00000092364" "ENSMUSG00000110099"
[44434

[45082] "ENSMUSG00000109326" "ENSMUSG00000051900" "ENSMUSG00000109578"
[45085] "ENSMUSG00000044141" "ENSMUSG00000108911" "ENSMUSG00000109113"
[45088] "ENSMUSG00000109304" "ENSMUSG00000030884" "ENSMUSG00000030887"
[45091] "ENSMUSG00000108935" "ENSMUSG00000097343" "ENSMUSG00000046096"
[45094] "ENSMUSG00000108830" "ENSMUSG00000030889" "ENSMUSG00000109392"
[45097] "ENSMUSG00000085118" "ENSMUSG00000109337" "ENSMUSG00000035064"
[45100] "ENSMUSG00000030880" "ENSMUSG00000030878" "ENSMUSG00000030877"
[45103] "ENSMUSG00000058050" "ENSMUSG00000088935" "ENSMUSG00000053358"
[45106] "ENSMUSG00000099280" "ENSMUSG00000109206" "ENSMUSG00000108954"
[45109] "ENSMUSG00000109036" "ENSMUSG00000035004" "ENSMUSG00000030876"
[45112] "ENSMUSG00000034990" "ENSMUSG00000108502" "ENSMUSG00000108697"
[45115] "ENSMUSG00000108627" "ENSMUSG00000108351" "ENSMUSG00000046321"
[45118] "ENSMUSG00000108409" "ENSMUSG00000108811" "ENSMUSG00000063317"
[45121] "ENSMUSG00000053081" "ENSMUSG00000000216" "ENSMUSG00000108410"
[45124

[45772] "ENSMUSG00000059119" "ENSMUSG00000098593" "ENSMUSG00000082561"
[45775] "ENSMUSG00000010755" "ENSMUSG00000086828" "ENSMUSG00000045362"
[45778] "ENSMUSG00000010751" "ENSMUSG00000109251" "ENSMUSG00000037613"
[45781] "ENSMUSG00000037606" "ENSMUSG00000050276" "ENSMUSG00000109421"
[45784] "ENSMUSG00000076944" "ENSMUSG00000048965" "ENSMUSG00000031090"
[45787] "ENSMUSG00000058454" "ENSMUSG00000031085" "ENSMUSG00000099321"
[45790] "ENSMUSG00000093221" "ENSMUSG00000109500" "ENSMUSG00000085745"
[45793] "ENSMUSG00000037541" "ENSMUSG00000085228" "ENSMUSG00000031078"
[45796] "ENSMUSG00000037519" "ENSMUSG00000073778" "ENSMUSG00000031077"
[45799] "ENSMUSG00000108851" "ENSMUSG00000031075" "ENSMUSG00000103394"
[45802] "ENSMUSG00000109009" "ENSMUSG00000109442" "ENSMUSG00000108874"
[45805] "ENSMUSG00000031074" "ENSMUSG00000050917" "ENSMUSG00000096999"
[45808] "ENSMUSG00000031073" "ENSMUSG00000108977" "ENSMUSG00000115783"
[45811] "ENSMUSG00000031072" "ENSMUSG00000070348" "ENSMUSG00000108967"
[45814

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [65]:
%%R
reactome_ensembl_ids <- intersect(ensembl2rxns.df$V1,ensembl_wo_ids)
print(reactome_ensembl_ids)


[5914] "ENSMUSG00000036940" "ENSMUSG00000036943" "ENSMUSG00000036957"
[5917] "ENSMUSG00000036960" "ENSMUSG00000036961" "ENSMUSG00000036977"
[5920] "ENSMUSG00000036980" "ENSMUSG00000036985" "ENSMUSG00000036986"
[5923] "ENSMUSG00000036992" "ENSMUSG00000037001" "ENSMUSG00000037005"
[5926] "ENSMUSG00000037007" "ENSMUSG00000037010" "ENSMUSG00000037012"
[5929] "ENSMUSG00000037014" "ENSMUSG00000037022" "ENSMUSG00000037025"
[5932] "ENSMUSG00000037032" "ENSMUSG00000037035" "ENSMUSG00000037049"
[5935] "ENSMUSG00000037053" "ENSMUSG00000037058" "ENSMUSG00000037089"
[5938] "ENSMUSG00000037098" "ENSMUSG00000037104" "ENSMUSG00000037110"
[5941] "ENSMUSG00000037111" "ENSMUSG00000037119" "ENSMUSG00000037130"
[5944] "ENSMUSG00000037140" "ENSMUSG00000037148" "ENSMUSG00000037152"
[5947] "ENSMUSG00000037157" "ENSMUSG00000037166" "ENSMUSG00000037171"
[5950] "ENSMUSG00000037174" "ENSMUSG00000037185" "ENSMUSG00000037197"
[5953] "ENSMUSG00000037204" "ENSMUSG00000037206" "ENSMUSG00000037211"
[5956] "ENSMUSG0000

[7078] "ENSMUSG00000045569" "ENSMUSG00000045573" "ENSMUSG00000045594"
[7081] "ENSMUSG00000045613" "ENSMUSG00000045625" "ENSMUSG00000045662"
[7084] "ENSMUSG00000045664" "ENSMUSG00000045671" "ENSMUSG00000045691"
[7087] "ENSMUSG00000045699" "ENSMUSG00000045708" "ENSMUSG00000045730"
[7090] "ENSMUSG00000045731" "ENSMUSG00000045733" "ENSMUSG00000045757"
[7093] "ENSMUSG00000045763" "ENSMUSG00000045795" "ENSMUSG00000045817"
[7096] "ENSMUSG00000045867" "ENSMUSG00000045871" "ENSMUSG00000045875"
[7099] "ENSMUSG00000045903" "ENSMUSG00000045948" "ENSMUSG00000045983"
[7102] "ENSMUSG00000045994" "ENSMUSG00000045996" "ENSMUSG00000046008"
[7105] "ENSMUSG00000046010" "ENSMUSG00000046027" "ENSMUSG00000046034"
[7108] "ENSMUSG00000046070" "ENSMUSG00000046079" "ENSMUSG00000046095"
[7111] "ENSMUSG00000046110" "ENSMUSG00000046139" "ENSMUSG00000046150"
[7114] "ENSMUSG00000046152" "ENSMUSG00000046159" "ENSMUSG00000046169"
[7117] "ENSMUSG00000046179" "ENSMUSG00000046185" "ENSMUSG00000046186"
[7120] "ENSMUSG00000

[7978] "ENSMUSG00000057280" "ENSMUSG00000057286" "ENSMUSG00000057315"
[7981] "ENSMUSG00000057321" "ENSMUSG00000057322" "ENSMUSG00000057329"
[7984] "ENSMUSG00000057337" "ENSMUSG00000057342" "ENSMUSG00000057346"
[7987] "ENSMUSG00000057367" "ENSMUSG00000057378" "ENSMUSG00000057388"
[7990] "ENSMUSG00000057396" "ENSMUSG00000057406" "ENSMUSG00000057409"
[7993] "ENSMUSG00000057421" "ENSMUSG00000057425" "ENSMUSG00000057439"
[7996] "ENSMUSG00000057440" "ENSMUSG00000057454" "ENSMUSG00000057469"
[7999] "ENSMUSG00000057522" "ENSMUSG00000057530" "ENSMUSG00000057531"
[8002] "ENSMUSG00000057551" "ENSMUSG00000057604" "ENSMUSG00000057614"
[8005] "ENSMUSG00000057650" "ENSMUSG00000057654" "ENSMUSG00000057666"
[8008] "ENSMUSG00000057667" "ENSMUSG00000057672" "ENSMUSG00000057674"
[8011] "ENSMUSG00000057691" "ENSMUSG00000057699" "ENSMUSG00000057722"
[8014] "ENSMUSG00000057723" "ENSMUSG00000057729" "ENSMUSG00000057738"
[8017] "ENSMUSG00000057751" "ENSMUSG00000057754" "ENSMUSG00000057788"
[8020] "ENSMUSG00000

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [66]:
%%R
saveRDS(reactome_ensembl_ids,file=paste(OUT_DIR,"reactome_ensembl_ids.Rds",sep=""))

In [67]:
%%R
GEO_model_training.df <- GEO_model_training.df[reactome_ensembl_ids,]
print(GEO_model_training.df)

                       V1     V2     V3      V4      V5     V6      V7      V8
ENSMUSG00000000001  49563 408439 341885 1063919   47611 117464 1434723  966419
ENSMUSG00000000028    532  10996  16406   56246     731   3559  113543   11145
ENSMUSG00000000049 910534   3593   2670   11882 2284180    430    4580 7034807
ENSMUSG00000000058   2570 179670   4762  187275    3109 316758    1026   71592
ENSMUSG00000000085   1550 658505  34278  261368   13648  81303   83069  109255
ENSMUSG00000000088  85341 765546 223223  967679  502683  48532 1378456  298381
ENSMUSG00000000120   1296  29006   5934   45957    9216   2378     693   40079
ENSMUSG00000000125      0  25450  19008     150       0    100   29474       0
ENSMUSG00000000126    150   7471   1734    8233     675  15465       0    5535
ENSMUSG00000000127   3542 137091  22970  101423   13902  27354   98159  133139
ENSMUSG00000000142   1894 278948 193714  206407    4373  67916  418808   52680
ENSMUSG00000000149  27381 859223  59975  251420   27

ENSMUSG00000000058 616243  13264  367602 152277   10102 145050  24142    0
ENSMUSG00000000085  19438  12971   55167 289562    6813 101066  56939 3270
ENSMUSG00000000088  38832 195113  375059 236124  192785 955557 257323 1603
ENSMUSG00000000120    370   2605   15674   9997    3616    796    798    0
ENSMUSG00000000125      0   6733   80952  43708       0   1021      0    0
ENSMUSG00000000126   6739    808   18036  60020    2908   6943  22004  276
ENSMUSG00000000127  35623  12006  334988 159001   17884  54247  19262 1432
ENSMUSG00000000142  48917  39787   24941 238313   12088   2694   9406 1046
ENSMUSG00000000149  66322  21257  415567 842800  249876  66981  45985    0
ENSMUSG00000000154   3495  45761   49103   3562   89539 189542    422   24
ENSMUSG00000000168  86968  39944  150693 541763   99992 261433 141138  527
ENSMUSG00000000171  69719 329903  454780 263674  252718 829586 162753  851
                    V113   V114   V115    V116  V117   V118    V119 V120   V121
ENSMUSG00000000001 2

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



  524   1014    2704  1415
ENSMUSG00000000049   5026   1157    231     301   952   15   3095 4051721   196
ENSMUSG00000000058 155699  71645  86922 1810514 19793  349  25628    6646 51966
ENSMUSG00000000085  96128  71980  13050  163372 81071 1643 106723   21597 20541
ENSMUSG00000000088 226230 277447 114416 4706645 74127 2926  77168  107613 67965
ENSMUSG00000000120  84666  77676  10219   13159 35285  247   5568   10894 17532
ENSMUSG00000000125   1042    170    150       0   876    0   3181      13 28841
ENSMUSG00000000126   2988  54506   1792   26520   735   48   2667    1582  2162
ENSMUSG00000000127 195229 113999  14819   82611 40594  381  42989   28448 22312
ENSMUSG00000000142 111341  63280  34659  143427 49016 1296  23743   20171 12067
ENSMUSG00000000149 111431 134555 208058 1724049 50060 1450  83722  331571  4989
ENSMUSG00000000154    400    180   1486    4759     0    0      0  145060  1339
ENSMUSG00000000168 324094 383564  61450 4522609 76712 1033  98521  113593 12423
ENSMUSG000000

ENSMUSG00000000125      0    375      0       0      0     51    791   15796
ENSMUSG00000000126   9518   7885   5170     396    981   2417   9833   48548
ENSMUSG00000000127  24709  63409  21691   11703 362004  11416 130763  102015
ENSMUSG00000000142  44143  21190  32689    5386   9327   7285  52901  204438
ENSMUSG00000000149  45140  33387 123245  151273 390816  52233  37153 1728886
ENSMUSG00000000154   3097     82  32649   54913   6745   5012  33952    5295
ENSMUSG00000000168  70840 229742  96235   51296 104305 222946  20219  334331
ENSMUSG00000000171  55622 451939  85138  152766 104363 185693  94487  797010
                     V496  V497    V498  V499    V500    V501   V502 V503
ENSMUSG00000000001 148030 61683  161155 18689  226837  381440 162773    0
ENSMUSG00000000028   2898 17915    3912   791   14751    3975   8183    0
ENSMUSG00000000049    294   148 4163937  1089    4973 3965599     67    0
ENSMUSG00000000058 502613  1068    4908  5966  115397    8551  37923    0
ENSMUSG0000000

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




ENSMUSG00000000028  34153  66519   356    7575    438   3924   1022  13473
ENSMUSG00000000049    112     73    44   92409    132   3105   2377    826
ENSMUSG00000000058   1632 206817 10581  219005   1217 162292  10741  55774
ENSMUSG00000000085    762  36382 20874  107118   3777 119653   9226  63110
ENSMUSG00000000088 338266 163929 29867 1127390 369285 441024  47681 238230
ENSMUSG00000000120   9299  34813  1096    1515  11989  15330    250  43754
ENSMUSG00000000125   1122  84031  1622       0      0  12038      0    361
ENSMUSG00000000126   2391   6069  2395    6242    193  49859    584  52630
ENSMUSG00000000127  44203  57838  4515   37711   2167  75757   7082  78469
ENSMUSG00000000142  11687  72844  7737    9312    147  76931   1905  54363
ENSMUSG00000000149    459 100023 47252  585265   2730 325655  48597  93559
ENSMUSG00000000154      0  12390   299 2184735      0   1197 109011    814
ENSMUSG00000000168 351804 113711 23680  714401 264077 404143  71421 335702
ENSMUSG00000000171 27803

ENSMUSG00000000049    3359    2716 1454101 2332122 54874    447    410 4572774
ENSMUSG00000000058  197796   96420    6448   22212   408 195709  57476    4739
ENSMUSG00000000085  819997   91296    3467   23331 10497  13523    301       0
ENSMUSG00000000088 1026635 1359002   81200  399345  6058  72893 272226   73960
ENSMUSG00000000120   32479    4792    2414   17083   590   1733   4162   42523
ENSMUSG00000000125   24832       0     163       0     0    102   1605       0
ENSMUSG00000000126   22565   47894     320     202   102   1727   5585     684
ENSMUSG00000000127  143343   21940   10458   23438 16672   9392  21746    6708
ENSMUSG00000000142  336867   21764    7111   12873  4807   7069   8770    8052
ENSMUSG00000000149  900045   90894  118963  126027  7635  40969  30748     153
ENSMUSG00000000154    2232     200   48412  216406  4791   3180    450     548
ENSMUSG00000000168 1217820  820706   54352  154758 14397 137639 329037   86719
ENSMUSG00000000171  748102  936196  139215  745312  

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 23005  308501  346746
ENSMUSG00000000028   21503  1153   18995  129713
ENSMUSG00000000049    1796    39     803  106480
ENSMUSG00000000058 3860570  1733 4508683   70729
ENSMUSG00000000085  261276 10139   25929   65715
ENSMUSG00000000088  216884  9346  168416 1536675
ENSMUSG00000000120    3270   889       0    2176
ENSMUSG00000000125     251     0       0   45363
ENSMUSG00000000126   72515   420    6311    4736
ENSMUSG00000000127  134479  3976   89470  176214
ENSMUSG00000000142  265574  2942   85715    6333
ENSMUSG00000000149  562392 19174  170239   33887
ENSMUSG00000000154   53342   110    8566   13384
ENSMUSG00000000168  239135 13125  325015  109090
ENSMUSG00000000171  454505 22725  429535  484328
 [ reached 'max' / getOption("max.print") -- omitted 9567 rows ]


In [68]:
%%R
saveRDS(GEO_model_training.df,file=paste(OUT_DIR,"GEO_model_training_df.Rds",sep=""))